In [19]:
### 데이터 및 모듈 로딩
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

file = '../data/공정데이터 추가자료.csv'
data = pd.read_csv(file)

In [20]:
### 데이터 증강 절차 (전체 데이터 대상)
# 불필요 컬럼(Unnamed: 12 -> 데이터 값이 없거나 0) 제거
data = data.drop(columns=['time', 'Unnamed: 12'])


# 모든 설정값이 동일한 s_temp_sv와 c_temp_sv컬럼 제거
data = data.drop(columns=['s_temp_sv', 'c_temp_sv'])


# 2차 미팅 결과 E_scr_sv가 8이 아닌 다른 값으로 셋팅된 것은 다른 제품에 대한 셋팅값으로 판단, E_scr_sv 값이 8인 행만 남기기
data = data[data['E_scr_sv'] == 8]


# E_scr_sv 값이 모두 8이므로 학습에 변화량을 제공하지 않아 삭제
data = data.drop(columns=['E_scr_sv'])


# E_scr_sv 컬럼 삭제 후 확인 시 n_temp_sv 컬럼이 모두 같은 값으로 통일되었기에 해당 컬럼을 삭제
data = data.drop(columns=['n_temp_sv'])


# E_scr_pv 컬럼의 고유값 확인시 8과 0으로 나옴 -> E_scr_pv 값이 8인 경우와 0인 경우로 데이터 나누기
data_8 = data[data['E_scr_pv'] == 8]
data_0 = data[data['E_scr_pv'] == 0]


## E_scr_pv 값이 0인 경우 k_rpm_pv가 모두 0이고, scale_pv값이 0 또는 600이상 값으로만 나옴
# scale_pv 값이 600 이상인 행은 기준이 되는 3과 너무 큰 차이가 나므로 측정 오류로 판단, 모두 삭제
data = data[data['scale_pv'] < 600]


# scale_pv 값이 4 이상인 행 또한 측정 오류로 판단, 모두 삭제
data = data[data['scale_pv'] < 4]


# c_temp_pv 값이 혼자 65.1인 날 E_scr_pv, k_rpm_pv, scale_pv값이 모두 0이므로 삭제, 남은 c_temp_pv 값이 모두 68 이상이므로 c_temp_pv 값이 68 미만인 행 삭제
data = data[data['c_temp_pv'] >= 68]


# 실제 칼날 회전수로 파악하는 방식 결정, k_rpm_sv 컬럼 제거
data = data.drop(columns=['k_rpm_sv'])


# E_scr_pv값이 0인 모든 케이스가  k_rpm_sv값이 50미만에 포함, 해당 값들은 scale_pv도 대부분 0 이므로 제외 -> 남은 E_scr_pv 모두 8이라 의미가 없으므로 E_scr_pv 컬럼 제거
data = data.drop(columns=['E_scr_pv'])


# k_rpm_pv값이 50미만 값들은 타겟이 3g인 값들에 대한 k_rpm_pv값 차이가 커 오류로 판단, scale_pv 값이 50 미만인 행 삭제
data = data[data['k_rpm_pv'] >= 50]


# scale_pv값이 0 인 경우 미측정, 나머지 0 초과 2 이하의 경우 측정오류로 판단, 제거
data = data[(data['scale_pv'] == 0) | (data['scale_pv'] > 2)]
### 모델 생성을 위한 데이터 증강 by KNN
# scale_pv가 0이 아닌 데이터와 0인 데이터로 분리
non_zero_data = data[data['scale_pv'] != 0]
zero_data = data[data['scale_pv'] == 0]


# KNN을 위한 피처와 타겟 설정 (scale_pv가 0이 아닌 데이터)
features = non_zero_data.drop(columns=['scale_pv'])
target = non_zero_data['scale_pv']


# 데이터 스케일링
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)
scaled_zero_features = scaler.transform(zero_data.drop(columns=['scale_pv']))


# KNN 모델 적합
knn = KNeighborsRegressor(n_neighbors=50) # 설정 이웃 수 2->1->5->10->100->50->25->50
knn.fit(scaled_features, target)


# 미측정 데이터에 대한 예측값 생성 (scale_pv가 0인 데이터, 소수점 아래 둘째자리까지)
predicted_scale_pv = np.round(knn.predict(scaled_zero_features), 2)


# 예측값을 미측정 데이터에 추가
zero_data.loc[:, 'scale_pv'] = predicted_scale_pv


# 원본 데이터와 새로 생성된 데이터를 결합하여 인덱스 기준으로 재정렬
data = pd.concat([non_zero_data, zero_data]).sort_index()
data = data.loc[data.index]

C:\Users\KDP\AppData\Local\Temp\ipykernel_12588\904320866.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zero_data.loc[:, 'scale_pv'] = predicted_scale_pv


In [21]:
### 데이터 전처리 및 상관계수 도출

## 각 temp의 분포 범위 확인
# c_temp_pv -> min : 68.0 ~ max : 71.9 
# n_temp_pv -> min : 65.7 ~ max : 71.2
# s_temp_pv -> min : 65.4 ~ max : 72.4


# 유사한 값을 선택할 범위 설정
tolerance = 0.1  # temp 분포의 5% 이내로 설정


# 최빈값 계산 (단일 최빈값 추출-> mode()[0])
c_temp_mode = data['c_temp_pv'].mode()[0]
n_temp_mode = data['n_temp_pv'].mode()[0]
s_temp_mode = data['s_temp_pv'].mode()[0]


# 필터링 조건 설정
filtered_data = data[
    (data['c_temp_pv'].between(c_temp_mode - tolerance, c_temp_mode + tolerance)) &
    (data['n_temp_pv'].between(n_temp_mode - tolerance, n_temp_mode + tolerance)) &
    (data['s_temp_pv'].between(s_temp_mode - tolerance, s_temp_mode + tolerance))
]
filtered_data.describe()


# 상관관계 계산 (피어슨)
correlation = filtered_data['k_rpm_pv'].corr(filtered_data['scale_pv'])
print(f'Correlation between k_rpm_pv and scale_pv: {correlation}')

Correlation between k_rpm_pv and scale_pv: -0.28282937375086886


In [22]:
### 변화량 감지 및 결론 도출
# scale_pv 0.05g 변동에 따른 k_rpm_pv 변화량 계산
filtered_data['scale_pv_diff'] = filtered_data['scale_pv'].diff()
filtered_data['k_rpm_pv_diff'] = filtered_data['k_rpm_pv'].diff()


# 특정 변동 범위 내 데이터 필터링 및 NaN값 삭제(diff()함수의 첫 값은 항상 NaN)
tolerance_diff = 0.05
subset_data = filtered_data[abs(filtered_data['scale_pv_diff']) <= tolerance_diff].dropna()


# 회귀 분석을 위한 데이터 준비
X = subset_data['scale_pv_diff'].values.reshape(-1, 1)
y = subset_data['k_rpm_pv_diff'].values


# 회귀 분석 모델 생성 및 적합
regression_model = LinearRegression()
regression_model.fit(X, y)


# 회귀 분석 결과 : 기울기와 편향 추출
slope = regression_model.coef_[0]
intercept = regression_model.intercept_

print(f'회귀 분석 결과: k_rpm_pv 변화량 = {slope:.4f} * scale_pv 변화량 + {intercept:.4f}')

print(f'--------------------------------------------------------------------')
# scale_pv가 0.05 변할 때 k_rpm_pv의 변화량 계산
scale_pv_change = 0.05
k_rpm_pv_change = slope * scale_pv_change + intercept

print(f'scale_pv가 {scale_pv_change} 변할 때 k_rpm_pv의 변화량: {k_rpm_pv_change:.4f}')

회귀 분석 결과: k_rpm_pv 변화량 = -19.2959 * scale_pv 변화량 + -0.0168
--------------------------------------------------------------------
scale_pv가 0.05 변할 때 k_rpm_pv의 변화량: -0.9816


C:\Users\KDP\AppData\Local\Temp\ipykernel_12588\2461385278.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['scale_pv_diff'] = filtered_data['scale_pv'].diff()
C:\Users\KDP\AppData\Local\Temp\ipykernel_12588\2461385278.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['k_rpm_pv_diff'] = filtered_data['k_rpm_pv'].diff()
